In [ ]:
# 상품 및 서비스 추천기법

# 추천 시스템 = 협업필터링 + 컨텐츠 기반 필터링 (예를들어 연관규칙분석)

# 협업필터링(CF) = 메모리 기반 CF + 모델 기반 CF
# 메모리 기반 CF = 사용자 기반(사용자간 유사성) + 아이템 기반 (아이템간 유사성)
# 모델 기반 = 잠재요인 모델 기반 (베이지안 네트워크, 군집분석, SVM, 딥러닝 등 다양한 머신러닝 모형을 통해 사용자의 평점을 예측하는 모형을 만드는 방식인데 그중 잠재요인 모델 기반 협업 필터링이 요새 주목을 받고 있음)



In [ ]:
##### 사용자 기반 협업 필터링 이론------------------------------------

### 사용자간 유사도 측정 방법

# 1. 평균제곱차이 유사도
# 사용자 사이의 유클리드 거리(직선거리)에 기반한 유사도 측정 방법이다
# 우선 평균제곱차이(MSD) 를 구하고 이 값이 0이 되는 경우에 대응하기 위해 1을 더하고 사용자간 유사도가 높을수록 값이 작아지도록 역수를 취해 평균제곱차이유사도를 구한다

# 2. 코사인 유사도
# 코사인 각도를 이용하여 두 사용자간의 유사한 정도를 측정, 측정값은 0~1 이며 1에 가까울수록 유사도가 높다

# 3. 피어슨 유사도
# 두 사용자간의 상관계수를 이용하여 유사도 측정, 0~1 사이값 나오며 1에 가까울소록 유사도 높음

### 과정 살펴보기

# 1. 사용자간 유사도 계산
# 두 사용자가 공통으로 평가한 상품에 대해서만 계산하기에 공통으로 평가한 상품의 수가 적을 경우 유사성이 과대평가될 수 있다, 공통평가상품 없으면 유사도 0

# 2. 유사도 높은 K명의 평점을 기반으로 예측 평점 계산


##### 사용자 기반 협업 필터링 실습------------------------------------

### 1. 데이터 가져오기
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')
# 1000명의 사용자가 1700 편의 영화에 대해 평가한 총 10만개의 평점

### 2. 데이터 확인
import pandas as pd
ratings = pd.DataFrame(data.raw_ratings, columns = ['user','item','rate','timestamp'])
ratings.head()

### 사용자 정보 출력
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv ...
#users 데이터프레임 >> user_id, age, sex, occupation, zip_code

### 영화 정보 출력
#movies 데이터 프레임 >> movie_id, title, release_date, video_release_date, imdb_url

### 3. 모형학습 및 예측--------------------------------------------------

from surprise.model_selection import train_test
from surprise import KNNBasic

## 데이터 분할
trainset, testset = train_test_split(data, test_size = 0.3, random_state = 0)

## 모형학습 및 예측
model = KNNBasic (name='cosine', user_base = True)
# name = 유사도측정방식, user_base True로 하면 사용자기반 False로 하면 아이템기반, k 유사도를 측정할 최대 이웃의 수 // 기본값은 40임
# min_k 유사도를 측정할 최소 이웃의 충분하지 않으면 이웃집합이 0으로 설정하며 이때 예측평점은 단순 전체 평균값이 된다, 여기서는 기본값 1을 사용했다
predictions = model.fit(trainset).test(testset)
predictions[:3]
#[Prediction(uid_'468 , iid='160 , r_ui=3.0, est=3.54635, details{'actual_k':4.0, 'was_impossible':False}), Prediction(),Prediction() ...]

## 모형 평가
from suprise import accuracy
rmse = accuarcy.rmse(predictions)

## 고객별 추천 영화 리스트 출력
def recommend(predictions, n, k) :
    print("----고객별 추천 영화 리스트------")
    # 1. n명의 고객ID 추출
    uids=[pred.uid for pred in predictions][:n]
    # 2. 고객별 추천 리스트 출력
    for uid in uids:
        # 고객이 관람하지 않는 영화 추출
        seen_movies = ratings[ratings.user==uid]['item'].tolist()
        total_movies = movies['movie_id'].tolist()
        unseen_movies = [movie for movie in total_movies if movie not in seen_movies]

        # k개의 미관람 영화에 대한 평점 예측
        pred = [model.predict(str(uid),str(item)) for item in unseen_movies]


